In [1]:
import sys,os
sys.path.append("..")
import django
django.setup()
import pandas as pd
from io import StringIO
import requests
import datetime

In [ ]:
class CrawlStockTdccTW:
    def __init__(self):
        self.target_name = "台股集保餘額資訊"
        self.sub_market = ["sii", "otc"]
    
    @classmethod
    def crawl_main(cls,file=False):
        if file is False:
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_1\
            0_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

            res = requests.get("https://smart.tdcc.com.tw/opendata/getOD.ashx?id=1-5", headers=headers)  
            df = pd.read_csv(StringIO(res.text))
        else:
            df=pd.read_csv(file)
        df=df.astype(str)
        df=df[~df['證券代號'].str.contains('YY|YE')]
        df=df.rename(columns={
            '證券代號' :'stock_id_fk','持股分級':'hold_class',
            '人數':'people','股數':'hold_num','占集保庫存數比例%':'hold_pt'
        })
        df['stock_id']=df['stock_id_fk']+'_'+df['hold_class']
        df=df[df['hold_class']!='16']
        df.iloc[:,2:6]=df.iloc[:,2:6].apply(lambda s: pd.to_numeric(s, errors="coerce"))
        df['date'] = df[df.columns[0]].apply(lambda s:datetime.datetime.strptime(s,'%Y%m%d'))
        df=df.drop(columns=df.columns[0])
        return df
    
import datetime
df=CrawlStockTdccTW.crawl_main()
df

In [ ]:
df.columns[0]

In [ ]:
df=pd.read_csv('/Users/benbilly3/Downloads/CSV/2020/0124.csv')
df.columns[0]